### FE621 - Homework #3

**Author**: Sid Bhatia

**Date**: April 5th, 2023

**Pledge**: I pledge my honor that I have abided by the Stevens Honor System.

**Professor**: Sveinn Olafsson

**TA**: Dong Woo Kim

#### Problem #1 (Monte Carlo Error)

Use Monte Carlo simulation to price a European call option in the Black-Scholes model with the following parameters: $S_0 = 100, \sigma = 0.30, r = 0.05, T = 1,$ and $K = 100$.

a. Use (exact) simulation based on the closed-form solution of geometric Brownian motion. Use $n = 100,000$ paths. 

Clearly describe the steps of your simulation procedure, and provide formulas for the Monte Carlo estimator and a corresponding 95% confidence interval. Report both the estimator and the confidence interval. Does the confidence interval contain the true price of the option?

##### Procedure

1. **Simulation of Stock Prices**: According to BSM, the stock process $S_t$ at future time $t$ is as follows:

$$S_t = S_0 \exp\{(r - \frac{1}{2}\sigma^2)T + \sigma \sqrt{T} W_T\}$$

where

- $S_0$ = initial stock price
- $r$ = rfr (e.g., 3-month UST)
- $\sigma$ = vol
- $T$ = time till maturity
- $W_T$ = standard BM $\sim \mathcal{N}(0, T)$

2. **Payoff Calculation**: For a call option, the payoff at maturity is $(S_T - K)_+$ where $K$ is the strike price. For puts, it's the converse $(K - S_T)_+$.

3. **MC Estimator**: The price of the option is the present value of the expected payoff under the risk-neutral measure $\mathbb{Q}$, which is estimated as the average of the discounted payoffs across all simulated paths:

$$P = e^{-rT} \mathbb{E}^Q[f(S_t)], \; \text{where} \; f \; \text{is the payoff function}. \\ \dots \\ \hat{C} = \exp\{(-rT)\} \frac{1}{n} \sum_{i = 1}^n f(S_t)$$

4. **CI**: The 95% confidence interval for the true option price is given by $$\hat{C} \pm z_{\alpha / 2} \cdot SE$$ 

where $\alpha = 0.05$ and $SE = \text{standard error}$. Therefore,

$$\hat{C} = 1.96 \cdot \frac{\sigma_{\hat{C}}}{\sqrt{n}}$$

where $\sigma_{\hat{C}}$ is the standard deviation of the stimulated payoffs.

5. **True Price Comparison**: The true price of the option can be calculated using the BSM closed-form solution. We compare the confidence interval obtained from the Monte Carlo simulation with the true price to see if it contains the true price.

In [16]:
import numpy as np
from scipy.stats import norm

def simulate_stock_prices(S_0: float, sigma: float, r: float, T: float, n: int) -> np.ndarray:
    """
    Simulate end stock prices using the closed-form solution of GBM.
    
    Parameters:
    - S0: Initial stock price
    - sigma: Volatility of the stock price
    - r: Risk-free interest rate
    - T: Time to maturity
    - n: Number of paths to simulate
    
    Returns:
    - A numpy array containing simulated end stock prices.
    """

    W_T = np.random.normal(0, 1, n)
    S_T = S_0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * W_T)
    return S_T

def monte_carlo_option_pricing(S_0: float, K: float, sigma: float, r: float, T: float, n: int) -> tuple:
    """
    Price a European call option using Monte Carlo simulation with geometric Brownian motion.
    
    Parameters:
    - S0: Initial stock price
    - K: Strike price
    - sigma: Volatility of the stock price
    - r: Risk-free interest rate
    - T: Time to maturity
    - n: Number of paths to simulate
    
    Returns:
    - The estimated option price and its 95% confidence interval as a tuple.
    """
    
    S_T = simulate_stock_prices(S_0, sigma, r, T, n)
    call_payoff = np.maximum(S_T - K, 0)

    option_price_estimate = np.exp(-r * T) * np.mean(call_payoff)

    standard_error = np.std(call_payoff) * np.exp(-r * T) / np.sqrt(n)

    confidence_interval = (option_price_estimate - 1.96 * standard_error, option_price_estimate + 1.96 * standard_error)
    
    return option_price_estimate, confidence_interval

# Parameters
S_0 = 100  # Initial stock price
sigma = 0.30  # Volatility
r = 0.05  # Risk-free rate
T = 1  # Time to maturity
K = 100  # Strike price
n = 100000  # Number of paths

# Running the Monte Carlo simulation.
option_price, confidence_interval = monte_carlo_option_pricing(S_0, K, sigma, r, T, n)
option_price, confidence_interval

(14.245282876176171, (14.106135160743374, 14.384430591608968))